In [1]:
import utils as keu
import pandas as pd

In [2]:
from transformers import ElectraTokenizer

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-generator")

In [2]:
raw_data = pd.read_csv('data/raw_book_info_list.csv',index_col=0)


raw_data.head(1)

,book_title,book_toc,book_intro,publisher
0,한 권으로 끝내는 메타버스 크리에이터,"['메타버스란', '왜 메타버스인가', '메타버스의 유형을 알아보자', '메타버스 ...",[],[]


### publisher

In [76]:
import ast

publisher = raw_data.publisher

# publisher 전처리
publisher_to_list = publisher.tolist()

# str type의 list를 다시 list type으로 변환
publisher_to_list = list(map(lambda x : ast.literal_eval(x),publisher_to_list))

# 비어있는 list 제거
publisher_to_list = [l for l in publisher_to_list if l ]

book_len = len(publisher_to_list)


publisher_to_list =  [list(filter(None,l)) for l in publisher_to_list if l ]

publisher_to_list = [j for i in publisher_to_list for j in i]

print(f'publisher column : {book_len}개 도서에서 {len(publisher_to_list)}개 문장 확보')

publisher column : 3749개 도서에서 75272개 문장 확보


In [78]:
df_publisher = pd.DataFrame(publisher_to_list)

df_publisher[0].str.len().sort_values(ascending=False)[:10]

63727    1939
39938    1467
66165    1034
3201     1003
8060      957
71202     913
24328     881
16703     867
16585     860
8253      773
Name: 0, dtype: int64

### Tokenizing 개수 체크

In [79]:
tokens = tokenizer(df_publisher[0].tolist())
tokens_num = [len(i) for i in tokens['input_ids']]
df_token_num = pd.DataFrame(tokens_num)

df_token_num = df_token_num.sort_values(by=0,ascending=False)

df_token_num.head(5)

,0
63727,770
39938,581
66165,473
3201,451
8060,446


### Publisher 저장

In [82]:
# Tokenizer 256개 이하 추가 
BM = df_token_num[df_token_num[0] <= 256].index

df_publisher_sorted = df_publisher[df_publisher.index.isin(BM)]

df_publisher_sorted = df_publisher_sorted.reset_index(drop=True)


## Eng to han
englist = pd.read_csv("preprocess/englist.csv")

lst = []
for i in df_publisher_sorted[0].tolist() :
    val = keu.trans_eng_to_han(words=i,englist=englist)

    lst.append(' '.join(val))


df_publisher_sorted = pd.DataFrame(lst)

# len 5개 이상만 포함
df_publisher_sorted = df_publisher_sorted[df_publisher_sorted[0].str.len() > 5]

df_publisher_sorted.to_csv('data/pre_book_publisher.csv')



### book_intro

In [4]:
book_intro = raw_data.book_intro

# publisher 전처리
book_intro_to_list = book_intro.tolist()

# str type의 list를 다시 list type으로 변환
book_intro_to_list = list(map(lambda x : ast.literal_eval(x),book_intro_to_list))

# 비어있는 list 제거
book_intro_to_list = [l for l in book_intro_to_list if l ]

book_len = len(book_intro_to_list)

book_intro_to_list =  [list(filter(None,l)) for l in book_intro_to_list if l ]

book_intro_to_list = [j for i in book_intro_to_list for j in i]

print(f'book_intro column : {book_len}개 도서에서 {len(book_intro_to_list)}개 문장 확보')

book_intro column : 2828개 도서에서 24039개 문장 확보


### Tokenizing 개수 체크

In [52]:
df_book_intro = pd.DataFrame(book_intro_to_list)

tokens = tokenizer(df_book_intro[0].tolist())
tokens_num = [len(i) for i in tokens['input_ids']]
df_token_num = pd.DataFrame(tokens_num)

df_token_num = df_token_num.sort_values(by=0,ascending=False)

df_token_num.head(10)

,0
20078,202
41,189
21169,153
21168,150
13393,146
...,...
11545,93
20180,93
9354,92
10851,92


### book_intro 저장

In [84]:
## Eng to han
englist = pd.read_csv("preprocess/englist.csv")

lst = []
for i in df_book_intro[0].tolist() :
    val = keu.trans_eng_to_han(words=i,englist=englist)

    lst.append(' '.join(val))


df_book_intro = pd.DataFrame(lst)

# len 5개 이상만 포함
df_book_intro = df_book_intro[df_book_intro[0].str.len() > 5]

df_book_intro.to_csv('data/pre_book_intro.csv')

### book_Toc

In [5]:
book_toc = raw_data.book_toc

# publisher 전처리
book_toc_to_list = book_toc.tolist()

# str type의 list를 다시 list type으로 변환
book_toc_to_list = list(map(lambda x : ast.literal_eval(x),book_toc_to_list))

# 비어있는 list 제거
book_toc_to_list = [l for l in book_toc_to_list if l ]

book_len = len(book_toc_to_list)

book_toc_to_list =  [list(filter(None,l)) for l in book_toc_to_list if l ]

book_toc_to_list = [j for i in book_toc_to_list for j in i]
book_toc_to_list = [i for i in book_toc_to_list if len(i) > 5]

print(f'book_toc column : {book_len}개 도서에서 {len(book_toc_to_list)}개 문장 확보')

book_toc column : 5043개 도서에서 495055개 문장 확보


### book_toc 저장

In [85]:
df_toc = pd.DataFrame(book_toc_to_list)

## Eng to han
englist = pd.read_csv("preprocess/englist.csv")

lst = []
for i in df_toc[0].tolist() :
    val = keu.trans_eng_to_han(words=i,englist=englist)

    lst.append(' '.join(val))


df_toc = pd.DataFrame(lst)

# len 5개 이상만 포함
df_toc = df_toc[df_toc[0].str.len() > 5]

# 문장 len
BM = df_toc[0].str.len().sort_values(ascending=False)
print('TOP 10 제거')
print(BM[:10])
df_toc[df_toc.index.isin(BM[11:].index)].to_csv('data/pre_book_toc.csv')

TOP 10 제거
451237    19948
57337     18113
281331    16584
63107     14962
434685      800
434678      630
434693      563
434695      535
434675      513
434687      475
Name: 0, dtype: int64


### Reply

In [5]:
# raw_reply = pd.read_csv('data/book_reply.csv',index_col=0)

# raw_reply = raw_reply[raw_reply['2'].isna() == False]['2'].tolist()

# # str type의 list를 다시 list type으로 변환
# book_reply_to_list = list(map(lambda x : ast.literal_eval(x),raw_reply))

# # 비어있는 list 제거
# book_reply_to_list = [l for l in book_reply_to_list if l ]

# book_len = len(book_reply_to_list)

# # 리스트 하나로 통일_1
# book_reply_to_list = [j for i in book_reply_to_list for j in i]

# # 리스트 하나로 통일_2
# book_reply_to_list = [j for i in book_reply_to_list for j in i]



# sentence_length = len(book_reply_to_list)

# print(f'book_reply column : {book_len}개 도서에서 {sentence_length}개 문장 확보')

book_reply column : 3549개 도서에서 102367개 문장 확보


### Book_intro + book_publisher + book_reply 종합

In [3]:
pre_book_intro = pd.read_csv('data/pre_book_intro.csv',index_col=0)
pre_book_publisher = pd.read_csv('data/pre_book_publisher.csv',index_col=0)
pre_book_reply = pd.read_csv('data/pre_book_reply.csv',index_col=0)

### Pre_book_total

In [16]:
pre_book_total = pd.concat([pre_book_intro,pre_book_publisher,pre_book_reply])

pre_book_total.head(3)

,0
0,이 책은 Do it 점프 투 파이썬 의 박응용 저자가 그동안 수많은 독자에게 받았던...
1,파이썬의 가장 큰 장점은 여러 분야에 걸쳐 방대한 라이브러리가 이미 만들어져 있다는...
2,하지만 어떤 라이브러리가 있는지 모른다면 제대로 쓸 수 없다


### Token 개수 세기

In [5]:
token_num = tokenizer(pre_book_total['0'].tolist())

Token indices sequence length is longer than the specified maximum sequence length for this model (815 > 512). Running this sequence through the model will result in indexing errors


In [26]:
token = [[i,len(i)] for i in token_num['input_ids']]

In [ ]:
df_token = pd.DataFrame(token)

df_token['sen'] = pre_book_total['0'].tolist()

df_token.columns = ['token','num','sen']


### 3 < token <256 이하 필터링

In [54]:
df_token = df_token[(df_token.num < 256) & (df_token.num > 3) ].sort_values(by='num',ascending=False)

df_token = df_token.reset_index(drop=True)

df_token.head(3)

,token,num,sen
0,"[2, 10698, 6537, 4073, 6248, 5277, 4704, 9770,...",251,책의 구성에 대해첫째 마당 스위프트와 만나기v 스위프트가 무엇인지 간략히 알아보자 ...
1,"[2, 28, 14370, 4279, 4031, 9577, 8800, 9108, 2...",250,8 할당하기 Chapter 9 태그 라벨 계정 Chapter 10 목표 달성을 위한...
2,"[2, 6663, 4234, 8075, 4073, 24612, 4319, 4219,...",249,시대의 흐름에 발맞추고 싶은 사람 구글 카드보드로 유투브의 도 영상만 보는 사람 새...


### Token 개수 256개 이하만 넣기

In [52]:
df_token.to_csv('data/pre_book_total_256.csv')

### Token 개수 128개 이하만 넣기

In [53]:
df_token[df_token['num'] <= 128].to_csv('data/pre_book_total_128.csv')